# Data Segmentation

## Tasks

- [x] Import `.wav` files.
- [x] Segment.
- [ ] Run downsampling protocol.
- [ ] 
- [ ] Export as `.npy`.
- [ ] Run Scipy.spectrum to obtain images.
    - [ ] Apply mel-cepstrum spacing.


In [110]:
# IMPORT BOX #
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import scipy.io

In [ ]:
# Constants
num_energy_buckets = 20

In [111]:
def segment_audio(FS, waves, pad=.1):
    num_samples = int(len(waves)/FS) # Assuming one per second.
    wave = (waves[:,0] + waves[:,1])*0.5
    sample_list = []
    
    for i in range(num_samples):
        wave_start = int(i*FS+pad*FS)
        wave_end = int((i+1)*FS-pad*FS)
        sample_list.append(wave[wave_start:wave_end])

    return sample_list

In [112]:
def get_energy(sample_list, num_buckets, normalize=True):
    energy_list = []
    
    for sample in sample_list:
        energy = sample*sample
        energy_list.append(energy)
    
    return downsample_X(np.asarray(energy_list), num_buckets, normalize=normalize)
        
def downsample_X(X_full_sample, dimensionality, debug=False, normalize=True):
    # Downsampling
    X = np.zeros( [len(X_full_sample), dimensionality] )

    step_size = int(len(X_full_sample[0])/dimensionality)

    for ind in range(len(X_full_sample)):
        cnt = 0
        for j in range(0, len(X_full_sample[ind])-step_size, step_size):
            X[ind][cnt] = np.mean(X_full_sample[ind][j:j+step_size])

            cnt += 1
            
    if normalize:
        for i in range(X.shape[0]):
            X[i,:] /= max(X[i,:])

    if debug:
        plt.title('Full Resolution Vector')
        plt.plot(X_full_sample[0])
        plt.show()
        plt.title('Down Sampled (Averaged) Vector (Dimensionality: {})'.format(dimensionality))
        plt.plot(X[0])

    return X

In [113]:
def get_energy_data(file_path, pad=0.4, output_dims=20, normalize=True):
    FS, audio = wavfile.read(file_path)
    
    output = segment_audio(FS, audio, pad=pad)
    output = get_energy(output, output_dims, normalize=True)
    
    return output

In [114]:
scratch_X = get_energy_data('raw_data/Scratches_01.wav', output_dims=num_energy_buckets)
taps_X = get_energy_data('raw_data/Taps_01.wav')
rose_X = get_energy_data('raw_data/Rose_01.wav')
silence_X = get_energy_data('raw_data/Silence_01.wav')

In [115]:
# Creating the dataset of X and Y: 

X = np.concatenate([scratch_X, taps_X, rose_X, silence_X])
Y = np.concatenate([np.ones([86,1]), 2*np.ones([86,1]), 3*np.ones([86,1]), 4*np.ones([86,1])])
len(X)

344

In [116]:
np.save('processed_data/X_energy.npy', X)
np.save('processed_data/Y_energy.npy', Y)